In [0]:
# Makes plots / images embedded in the file
%matplotlib inline

In [0]:
from zipfile import ZipFile
from urllib.request import urlopen
import numpy as np

URL = 'https://uwmadison-cs534-term-project-f2018-cdh.github.io/assets/datasets/items.zip'

url = urlopen(URL)
output = open('items.zip','wb')
output.write(url.read())
output.close()

zip_ref = ZipFile('items.zip', 'r')
zip_ref.extractall('./dataset')
zip_ref.close()

In [0]:
# Import data and keras layers

from keras import backend as k
from keras.models import Model, Sequential
from keras.layers import Input, Conv2D, UpSampling2D, Dense, Reshape
from keras.layers import Flatten, Dropout, BatchNormalization, Activation
from keras.layers.advanced_activations import LeakyReLU
from keras.optimizers import Adam
from tqdm import tqdm
import matplotlib.pyplot as plt

In [0]:
import os

files = os.listdir('./dataset/items')
for name in files:
    print(name)

In [0]:
x1_raw = np.load('./dataset/items/items_image.npy')
y1_raw = np.load('./dataset/items/items_label.npy')

x1_test = np.array([])
y1_test = np.array([])

#Optimizer
adam = Adam(lr=2e-4, beta_1=0.5)

In [0]:
filterLbls = ['items_weapon','items_weapons','items_weapon_ranged','items_weapon_artefact']
#'items_wand','items_staff','items_rod'

y1_train = []

firstTime = True
for i in range(y1_raw.shape[0]):
  
  if y1_raw[i] in filterLbls:
    y1_train.append(y1_raw[i])
    
    if firstTime:
      firstTime = False
      raw = x1_raw[i,:,:,:]
      x1_train = np.reshape(raw,(1,raw.shape[0],raw.shape[1],raw.shape[2]))
    else:
      raw = x1_raw[i,:,:,:]
      raw = np.reshape(raw,(1,raw.shape[0],raw.shape[1],raw.shape[2]))
      x1_train = np.concatenate((x1_train,raw), axis=0)
    
y1_train = np.array(y1_train)


In [0]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
  featurewise_center = True,
  featurewise_std_normalization = True,
  rotation_range = 60,
  width_shift_range = 0.2,
  height_shift_range = 0.2,
  shear_range = 0.2,
  zoom_range = 0.2
)

datagen.fit(x1_train)

In [0]:
# Creating the generator
generator = Sequential()
generator.add(Dense(128*8*8, input_dim = 100))
generator.add(Activation('relu'))
generator.add(BatchNormalization())
generator.add(Reshape((8, 8, 128)))
generator.add(UpSampling2D((2,2)))
generator.add(Conv2D(64, kernel_size = (5, 5), padding = 'same'))
generator.add(Activation('relu'))
generator.add(BatchNormalization())
generator.add(UpSampling2D((2,2)))
generator.add(Conv2D(4, kernel_size = (5, 5), padding = 'same'))
generator.add(Activation('tanh'))

print(generator.summary())
generator.compile(loss = 'binary_crossentropy', optimizer = adam)  

In [0]:
# Creating the discriminator
discriminator = Sequential()
discriminator.add(Conv2D(64, kernel_size = (5, 5), strides = (2, 2), padding = 'same', input_shape = (32, 32, 4)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Conv2D(128, kernel_size = (5, 5), strides = (2, 2), padding = 'same'))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Flatten())
discriminator.add(Dropout(0.5))
discriminator.add(Dense(1, activation = 'sigmoid'))
print(discriminator.summary())
discriminator.compile(loss = 'binary_crossentropy', optimizer = adam) 

In [0]:
# Create the GAN
discriminator.trainable = False
gan = Sequential()
gan.add(generator)
gan.add(discriminator)
gan.compile(loss = 'binary_crossentropy', optimizer = adam)

In [0]:
# Visualize generated data
def plotGeneratedImages():
  generated_images = generator.predict(noise)
  fig = plt.figure(figsize=(9, 9))        
  for i in range(9 if generated_images.shape[0] > 9 else generated_images.shape[0]):
    plt.subplot(3, 3, i+1)
    img = np.clip(generated_images[i, :],0,1)
    img = img.reshape((32, 32, 4))
    plt.tight_layout()
    plt.imshow(img)
    plt.axis('off')

In [0]:
epochs = 20
batch_size = 16

batch_count = int(x1_train.shape[0] / batch_size)

iterator = datagen.flow(x1_train, batch_size=batch_size)

for epoch in range(epochs):

    for index in tqdm(range(batch_count)):
        
        # Real images
        #image_batch = x1_train[index * batch_size: (index + 1) * batch_size]
        image_batch = iterator.next()
        
        # Get initial random input for generator
        noise = np.random.uniform(-1, 1, size=[batch_size, 100])
        # Generating images
        generated_data = generator.predict_on_batch(noise)

        X = np.concatenate([generated_data, image_batch])
        y = np.zeros(batch_size + image_batch.shape[0])
        y[:image_batch.shape[0]] = 1

        # Train discriminator
        discriminator.trainable = True
        discriminator.train_on_batch(x=X, y=y)

        # Train generator 
        y = np.zeros(batch_size)
        discriminator.trainable = False
        gan.train_on_batch(x=noise, y=y)
        
        img = generated_images = generator.predict(noise)   
        
    
plotGeneratedImages()


In [0]:
noise = np.random.uniform(-1, 1, size=[9, 100])
plotGeneratedImages()

In [0]:
from google.colab import files
import matplotlib.image as mpimg
import uuid

images = 20
noise = np.random.uniform(-1, 1, size=[images, 100])
generated_images = generator.predict(noise)    
for i in range(images):
  name = str(uuid.uuid4().hex) + '.png'
  img = np.reshape(np.clip(generated_images[i, :],0,1),(32, 32, 4))
  mpimg.imsave(name, img)             
  files.download(name)
